# Strands Agent with AgentCore Memory Tutorial using Hooks

## Overview

This tutorial demonstrates how to build an intelligent personal assistant using Strands agents integrated with AgentCore Memory through hooks. The agent maintains conversation context and learns from interactions to provide personalized responses.

## Tutorial Details

**Use Case**: Math Assistant

| Information         | Details                                                                          |
|:--------------------|:---------------------------------------------------------------------------------|
| Tutorial type       | Long term Conversational                                                         |
| Agent type          | Math Assistant                                                                   |
| Agentic Framework   | Strands Agents                                                                   |
| LLM model           | Anthropic Claude Sonnet 3.7                                                      |
| Tutorial components | AgentCore Summary Strategy for Memory, Hooks for storing and retrieving Memory   |
| Example complexity  | Intermediate                                                                     |


You'll learn to:
- Set up AgentCore Memory with conversation summaries
- Create memory hooks for automatic storage and retrieval
- Build a Strands agent with persistent memory
- Test memory functionality across conversations

### Scenario Context

In this example you'll create a Math Assistant example where you'd store summaries of the previous conversations. 
Key features of this example:
- **Automatic Memory Storage**: Conversations are automatically saved
- **Context Retrieval**: Previous conversations inform current responses
- **Summary Generation**: Key information is extracted and summarized
- **Tool Integration**: Calculator tool for mathematical operations

## Architecture
<div style="text-align:left">
    <img src="architecture.png" width="65%" />
</div>


## Prerequisites

To execute this tutorial you will need:
- Python 3.10+
- AWS credentials with Amazon Bedrock AgentCore Memory permissions
- Amazon Bedrock AgentCore SDK

## Step 1: Environment set up
Let's begin importing all the necessary libraries and defining the clients to make this notebook work.

In [ ]:
!pip install -qr requirements.txt

In [ ]:
from bedrock_agentcore.memory import MemoryClient
from bedrock_agentcore.memory.constants import StrategyType

In [ ]:
import os
import logging
from strands import Agent
from datetime import datetime
from strands_tools import calculator
from strands.hooks import AfterInvocationEvent, HookProvider, HookRegistry, MessageAddedEvent

# Setup logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")
logger = logging.getLogger("memory-tutorial")

# Configuration - replace with your values
REGION = os.getenv('AWS_REGION', 'us-west-2')
ROLE_ARN = "<<INSERT-YOUR-IAM-ROLE>>"
ACTOR_ID = f"actor-{datetime.now().strftime('%Y%m%d%H%M%S')}"
SESSION_ID = f"tutorial-{datetime.now().strftime('%Y%m%d%H%M%S')}"

## Step 2: Create Memory Resource

In this step, we're creating our memory resource with a summary strategy. This resource will store and organize our conversation data. The strategy we're defining will automatically generate summaries of conversations and store them in organized namespaces.

Firstly, lets create a custom prompt for the math assistant.


In [ ]:
CUSTOM_PROMPT = """
Your task is to extract math learning data from the user's conversations. You store the progress of the user in a memory system to understand their math level and help them progress.

You are tasked with analyzing conversations to extract the user's math learning patterns. You'll be analyzing two sets of data: 

<past_conversation> 
[Past conversations between the user and math tutor will be placed here for context] 
</past_conversation> 

<current_conversation> 
[The current conversation between the user and math tutor will be placed here] 
</current_conversation> 

Your job is to identify and categorize the user's math learning profile:
- Extract the user's current math level from problems they solve correctly/incorrectly
- Extract the user's preferred learning style from how they ask questions and respond to explanations
- Extract topic strengths and weaknesses from their performance patterns
- Track learning progress and identify areas needing reinforcement
"""

In [ ]:
from botocore.exceptions import ClientError

# Initialize Memory Client
client = MemoryClient(region_name=REGION)
memory_name = "MathAssistant"
# Define memory strategy for conversation summaries
strategies = [
    {
        StrategyType.CUSTOM.value: {
            "name": "CustomSemanticMemory",
            "description": "Captures facts from conversations",
            "namespaces": ["/students/math/{actorId}"],
            "configuration" : {
                "semanticOverride" : {
                    "extraction" : {
                        "modelId" : "anthropic.claude-3-5-sonnet-20241022-v2:0",
                        "appendToPrompt": CUSTOM_PROMPT
                    }
                },
    }}}
]

# Create memory resource
try:
    memory = client.create_memory_and_wait(
        name=memory_name,
        strategies=strategies, # Use the defined long term strategies
        description="Memory for tutorial agent",
        event_expiry_days=30,
        memory_execution_role_arn=ROLE_ARN,
    )
    memory_id = memory['id']
    logger.info(f"✅ Created memory: {memory_id}")
except ClientError as e:
    if e.response['Error']['Code'] == 'ValidationException' and "already exists" in str(e):
        # If memory already exists, retrieve its ID
        memories = client.list_memories()
        memory_id = next((m['id'] for m in memories if m['id'].startswith(memory_name)), None)
        logger.info(f"Memory already exists. Using existing memory ID: {memory_id}")
except Exception as e:
    # Handle any errors during memory creation
    logger.info(f"❌ ERROR: {e}")
    import traceback
    traceback.print_exc()
    # Cleanup on error - delete the memory if it was partially created
    if memory_id:
        try:
            client.delete_memory_and_wait(memory_id=memory_id)
            logger.info(f"Cleaned up memory: {memory_id}")
        except Exception as cleanup_error:
            logger.info(f"Failed to clean up memory: {cleanup_error}")

## Step 3: Create Memory Hook Provider

This step defines our custom `MemoryHookProvider` class that automates memory operations. Hooks are special functions that run at specific points in an agent's execution lifecycle. The memory hook we're creating serves two primary functions:

1. **Retrieve Memories**: Automatically fetches relevant past conversations when a user sends a message
2. **Save Memories**: Stores new conversations after the agent responds

This creates a seamless memory experience without manual management.

In [ ]:
class MemoryHookProvider(HookProvider):
    """Hook provider for automatic memory management"""
    
    def __init__(self, memory_id: str, client: MemoryClient, actor_id: str, session_id: str):
        self.memory_id = memory_id
        self.client = client
        self.actor_id = actor_id
        self.session_id = session_id
        self.namespace = f"/students/math/{self.actor_id}"
    
    def retrieve_memories(self, event: MessageAddedEvent):
        """Retrieve relevant memories before processing user message"""
        messages = event.agent.messages
        if messages[-1]["role"] == "user" and "toolResult" not in messages[-1]["content"][0]:
            user_message = messages[-1]["content"][0].get("text", "")
            
            try:
                # Retrieve relevant memories
                memories = self.client.retrieve_memories(
                    memory_id=self.memory_id,
                    namespace=self.namespace,
                    query=user_message
                )
                
                # Extract memory content
                memory_context = []
                for memory in memories:
                    if isinstance(memory, dict):
                        content = memory.get('content', {})
                        if isinstance(content, dict):
                            text = content.get('text', '').strip()
                            if text:
                                memory_context.append(text)
                
                # Inject memories into user message
                if memory_context:
                    context_text = "\n".join(memory_context)
                    original_text = messages[-1]["content"][0].get("text", "")
                    messages[-1]["content"][0]["text"] = (
                        f"{original_text}\n\nPrevious context: {context_text}"
                    )
                    logger.info(f"Retrieved {len(memory_context)} memories")
                    
            except Exception as e:
                logger.error(f"Failed to retrieve memories: {e}")
    
    def save_memories(self, event: AfterInvocationEvent):
        """Save conversation after agent response"""
        try:
            messages = event.agent.messages
            if len(messages) >= 2 and messages[-1]["role"] == "assistant":
                # Get last user and assistant messages
                user_msg = None
                assistant_msg = None
                
                for msg in reversed(messages):
                    if msg["role"] == "assistant" and not assistant_msg:
                        assistant_msg = msg["content"][0]["text"]
                    elif msg["role"] == "user" and not user_msg and "toolResult" not in msg["content"][0]:
                        user_msg = msg["content"][0]["text"]
                        break
                
                if user_msg and assistant_msg:
                    # Save conversation
                    self.client.create_event(
                        memory_id=self.memory_id,
                        actor_id=self.actor_id,
                        session_id=self.session_id,
                        messages=[(user_msg, "USER"), (assistant_msg, "ASSISTANT")]
                    )
                    logger.info("Saved conversation to memory")
                    
        except Exception as e:
            logger.error(f"Failed to save memories: {e}")
    
    def register_hooks(self, registry: HookRegistry) -> None:
        """Register memory hooks"""
        registry.add_callback(MessageAddedEvent, self.retrieve_memories)
        registry.add_callback(AfterInvocationEvent, self.save_memories)
        logger.info("Memory hooks registered")

## Step 4: Create Agent with Memory

Now we're creating our Strands agent and connecting it with our memory hook provider. This agent will have two key capabilities:

1. **Memory Integration**: The memory hooks we created will enable automatic context retrieval
2. **Calculator Tool**: The agent can perform mathematical operations when needed

This combination creates a personal assistant that both remembers past interactions and can perform useful calculations.

In [ ]:
# Create memory hook provider
memory_hooks = MemoryHookProvider(
    memory_id=memory_id,
    client=client,
    actor_id=ACTOR_ID,
    session_id=SESSION_ID
)

# Create agent with memory hooks and calculator tool
agent = Agent(
    hooks=[memory_hooks],
    tools=[calculator],
    system_prompt="You are a helpful personal math tutor. You assist users in solving math problems and provide personalized assistance."
)

print("✅ Agent created with memory hooks.")

**We have our agent set up ! Let's test it now.**

## Test Memory Functionality

In this section, we'll test the agent's memory capabilities through a series of interactions. We'll observe how the agent builds context over time and recalls previous interactions.

First, let's introduce ourselves to the agent and ask a math question:

In [ ]:
# First interaction - introduce yourself
response1 = agent("Hi, I'm John and I just enrolled in Discrete Math course. Help me solve this: How many ways can I arrange 5 books on a shelf?")
print(f"Agent: {response1}")

Let's give the agent another calculation task:

In [ ]:
# Second interaction - another calculation
response2 = agent("I learn better with step-by-step explanation with example questions. Can you explain modular arithmetic? What's 17 mod 5?")
print(f"Agent: {response2}")

Now, let's see if the agent remembers who we are.

**Note:** Give a ~20 sec pause here to allow some time for the memory to be extracted, consolidated and stored.

In [ ]:
# Third interaction - test memory recall
response3 = agent("I got that right! What's the immediate next step that I should study after modular arithmetic?")
print(f"Agent: {response3}")

Finally, let's check if the agent remembers our calculation history:

In [ ]:
# Fourth interaction - test context awareness
response4 = agent("This is too hard, can we try something easier?")
print(f"Agent: {response4}")

### Verify Memory Storage

As a final step, we'll verify that our conversations have been properly stored in AgentCore Memory. This demonstrates that the memory hooks are working correctly and the agent can access this information in future interactions.

In [ ]:
# Check stored memories
try:
    memories = client.retrieve_memories(
        memory_id=memory_id,
        namespace=f"/students/math/{ACTOR_ID}",
        query="mathematics calculations"
    )
    
    print(f"\n📚 Found {len(memories)} memories:")
    for i, memory in enumerate(memories, 1):
        if isinstance(memory, dict):
            content = memory.get('content', {})
            if isinstance(content, dict):
                text = content.get('text', '')[:200] + "..."
                print(f"{i}. {text}")
                
except Exception as e:
    print(f"Error retrieving memories: {e}")

Tutorial completed! 🎉

Key takeaways:
- Memory hooks automatically store and retrieve conversation context
- Agents can maintain state across multiple interactions
- AgentCore Memory provides semantic search for relevant context
- Tools can be combined with memory for enhanced functionality

## Clean Up

### Optional: Delete Memory Resource

After completing the tutorial, you may want to delete the memory resource to avoid incurring unnecessary costs. The following code is provided for cleanup but is commented out by default.

In [ ]:
# Uncomment to delete the memory resource
# try:
#     client.delete_memory_and_wait(memory_id=memory_id)
#     print(f"✅ Deleted memory resource: {memory_id}")
# except Exception as e:
#     print(f"Error deleting memory: {e}")